In [106]:
import os
import pandas as pd
import numpy as np
import time
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.common.by import By

## 1) Tor-Selenium Setup

In [107]:
torexe = os.popen(r'C:\Users\admin\Desktop\Tor Browser\Browser\TorBrowser\Tor\tor.exe')

firefoxBinary = r"C:\Users\admin\Desktop\Tor Browser\Browser\firefox.exe"
geckodriverPath = r"C:\Users\admin\Desktop\Tor Browser\Browser\geckodriver.exe"
proxyIP = "127.0.0.1"
proxyPort = 9150

binary = FirefoxBinary(firefoxBinary)

proxy_settings = {
    "network.proxy.type": 1,
    "network.proxy.socks": proxyIP,
    "network.proxy.socks_port": proxyPort,
    "network.proxy.socks_remote_dns": False,
    "extensions.torlauncher.start_tor": True
}

profile = FirefoxProfile(r'C:\Users\admin\Desktop\Tor Browser\Browser\TorBrowser\Data\Browser\profile.default')
profile.set_preference('extensions.torlauncher.start_tor', True) # bypass tor connection page
profile.set_preference('network.proxy.type', 1)
profile.set_preference('network.proxy.socks', '127.0.0.1')
profile.set_preference('network.proxy.socks_port', 9150)
profile.set_preference("network.proxy.socks_remote_dns", True)
profile.update_preferences()

driver = webdriver.Firefox(executable_path=geckodriverPath, 
                           firefox_binary=binary, 
                           firefox_profile=profile)

# Wait till your tor browser is connected to the tor network before executing this
driver.get('http://zohlm7ahjwegcedoz7lrdrti7bvpofymcayotp744qhx6gjmxbuo2yid.onion/')
    
time.sleep(10)
    
SCROLL_PAUSE_TIME = 10 # as the Tor browser is slow, might need to increase this accordingly

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height



In [149]:
companyNames = []
companyDescription = []
companyLocation = []
noClicks = []
dateVictimised = []
status = []
action = []


cards = driver.find_elements_by_class_name('cls_record')

for card in cards:

    companyNameElement = card.find_element_by_xpath(".//div[@class='cls_recordTop']")
    companyNameText = companyNameElement.text
    companyNames.append(companyNameText)
    
    
    companyDescriptionElement = card.find_element_by_xpath(".//div[@class='cls_recordMiddle']")
    companyDescriptionText = companyDescriptionElement.text
    companyDescription.append(companyDescriptionText)
    
    
    noClicksElement = card.find_element_by_xpath(".//div[@class='cls_recordBottom']/div[@class='cls_recordBottomElement']/div[@class='cls_headerSmall']")
    noClicksText = noClicksElement.text
    noClicks.append(noClicksText)
    
    
    statusElement = card.find_element_by_xpath("(.//div[@class='cls_recordBottom']/div)[2]")
    statusText = statusElement.text
    
    status_index = 8
    status_text = statusText[status_index::]
    status.append(status_text)
    
    
    actionElement = card.find_element_by_xpath("(.//div[@class='cls_recordBottom']/div)[3]")
    actionText = actionElement.text
    
    action_index = 8
    action_text = actionText[action_index::]
    action.append(action_text)
    
    
    dateVictimisedElement = card.find_element_by_xpath("(.//div[@class='cls_recordBottom']/div)[4]")
    dateVictimisedText = dateVictimisedElement.text
    
    date_index = 13
    date_text = dateVictimisedText[date_index::]
    dateVictimised.append(date_text)


#driver.quit()

## 3) Clean data + Extract to CSV/Excel format


Requirements for data:  
- Date of victim company within 2023

In [150]:
df = pd.DataFrame({"Company": companyNames, "Company Description": companyDescription, "Date Victimised": dateVictimised, "No of views": noClicks, "Status": status, "Action": action})
df



,Company,Company Description,Date Victimised,No of views,Status,Action
0,Van Oirschot,https://www.vanoirschot.be/,30/09/2023,2076,EVIDENCE,Encrypted
1,Hawkins Delafield Wood,https://www.hawkins.com/,03/09/2023,16949,EVIDENCE,Encrypted
2,Radley and Co,https://www.radleylondon.com/,29/08/2023,26442,EVIDENCE,Encrypted
3,SAC Finance,https://www.sacfinance.com/,03/06/2023,33011,EVIDENCE,Leak
4,Low Keng Huat,https://www.lkhs.com.sg/,02/07/2023,40381,EVIDENCE,Encrypted
5,Customer Elation - Business Information,https://www.customerelation.com/,01/06/2023,83491,EVIDENCE,Leak
6,Prada Gayoso,https://www.pradagayoso.com/,08/06/2023,132833,EVIDENCE,Encrypted
7,Roberto Verino Difusion,https://www.robertoverino.com/,04/05/2023,108505,DISCLOSED,Encrypted
8,Wison Engineering,https://www.wison.com/,03/06/2023,86601,DISCLOSED,Encrypted
9,Mission Community Hospital,https://www.mchonline.org/,29/04/2023,148457,EVIDENCE,Encrypted


In [152]:
df.to_csv("RansomHouse.csv")

## 3) Scraping Glassdoor for more company information

More information we need:  
- Industry  
- Geography  
  - Country  

Additional information we can scrape (the more we have, the more insights we can infer):
- Company Website

In [ ]:
# pip install webdriver-manager
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()

In [ ]:
import requests

def location_to_country(location):
    base_url = "https://nominatim.openstreetmap.org/search"
    
    params = {
        "addressdetails": 1,
        "q": location,
        "format": "jsonv2",
        "limit": 1
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    if data:
        country = data[0].get("address", {}).get("country", "")
        return country

    return np.nan

In [ ]:
# Using LinkedIn

companyWebsite_list = []
industry_list = []
headquarter_list = []
country_list = []

wait = WebDriverWait(driver, 5)

for company in companyNames:
    try:
        driver.get('https://www.google.com')
        search = driver.find_element(By.NAME, 'q')
        search.send_keys(company + ' linkedin')
        search.send_keys(Keys.RETURN)
        linkedin_page = driver.find_element_by_tag_name('h3') # clicking the first search result
        time.sleep(2) # to limit rate of requests to linkedin
        linkedin_page.click()

        if "linkedin" in driver.current_url:

            loaded = wait.until(EC.presence_of_element_located((By.ID, "main-content")))

            # Check if the scraped company name is even in the linkedin page (sometimes the company data is not in linkedin; false positive)
            # try:
            #     linkedin_companyName_element = driver.find_element_by_xpath("//h1[@class='top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0']")
            #     linkedin_companyName = linkedin_companyName_element.text
            # except:
            #     print(company)
            #     continue

            # if not substring_included(company, linkedin_companyName):
            #     print(company)
            #     continue

            # bypass sign in modal
            try:
                sign_in_button_element = driver.find_element_by_xpath("//button[@class='sign-in-modal__outlet-btn cursor-pointer btn-md btn-primary']")
                sign_in_button_element.click()
                dismiss_button_element = driver.find_element_by_xpath("//button[@class='modal__dismiss btn-tertiary h-[40px] w-[40px] p-0 rounded-full indent-0 sign-in-modal__dismiss absolute right-0 cursor-pointer m-[20px]']")
                dismiss_button_element.click()
            except:
                pass

            CLASS = ".//dd[@class='font-sans text-md text-color-text break-words overflow-hidden']"

            try:
                website_element = driver.find_element_by_css_selector('[data-test-id="about-us__website"]')
                website_text_element = website_element.find_element_by_xpath(CLASS)
                website_text = website_text_element.text
            except Exception as e:
                website_text = np.nan
                print(e)
            companyWebsite_list.append(website_text)

            try:
                industry_element = driver.find_element_by_css_selector('[data-test-id="about-us__industry"]')
                industry_text_element = industry_element.find_element_by_xpath(CLASS)
                industry_text = industry_text_element.text
            except Exception as e:
                industry_text = np.nan
                print(e)
            industry_list.append(industry_text)
            
            try:
                headquarters_element = driver.find_element_by_css_selector('[data-test-id="about-us__headquarters"]')
                headquarters_text_element = headquarters_element.find_element_by_xpath(CLASS)
                headquarters_text = headquarters_text_element.text

                country = location_to_country(headquarters_text)
            except Exception as e:
                headquarters_text = np.nan
                country = np.nan
                print(e)
            headquarter_list.append(headquarters_text)
            country_list.append(country)

        else:
            companyWebsite_list.append(np.nan)
            industry_list.append(np.nan)
            headquarter_list.append(np.nan)
            country_list.append(np.nan)

    except Exception as e:
        companyWebsite_list.append(np.nan)
        industry_list.append(np.nan)
        headquarter_list.append(np.nan)
        country_list.append(np.nan)
        print(e)
    number += 1
    
driver.quit()

In [ ]:
df["Company Website"] = companyWebsite_list
df["Industry"] = industry_list
df["Location"] = headquarter_list
df["Country"] = country_list
df

In [ ]:
df.to_csv("Medusa_Linkedin_Data.csv")